<a href="https://colab.research.google.com/github/McSloats/PAM-Backdoor/blob/master/Indep_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import csv
import pandas as pd
import scipy
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Dense
from sklearn import preprocessing
from keras import optimizers as op
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.callbacks import EarlyStopping


In [0]:
##IMPORT FILE FOR DATA EXTRACTION
benign = pd.read_csv("/content/drive/My Drive/ben.csv")
malicious = pd.read_csv("/content/drive/My Drive/mal.csv")
#CREATE SINGLE DATAFRAME FOR BOTH THE BENIGN AND MALICIOUS DATA WITH MIXED ROWS
benign['label'] = 1
malicious['label'] = 0
benign = benign.sample(frac=1,random_state=1)
malicious = malicious.sample(frac=1,random_state=1)
data = [benign,malicious]
data = pd.concat(data)
malicious = None
benign = None

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
print(data)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) ##ADJUST 'test_size' ACCORDINGLY 
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [0]:
def rnn(x_train, y_train, x_test, y_test):
    #KERAS MODEL    
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = x_train.shape[1]
    #add model layers
    model.add(Embedding(128, 64,input_length=n_cols))
    model.add(LSTM(128,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
   
    #Optimizer
    opt = op.Adam()
    #Compile model
    model.compile(opt, loss='binary_crossentropy', 
                  metrics=['accuracy'])
   
    #TRAINING
    #Put this var in callbacks to save model after each epoch
    #checkpointer = ModelCheckpoint('model/model-{epoch:02d}.hdf5', verbose=1)

    model.fit(x_train, y_train, batch_size=512, 
              validation_split=0.2, epochs=10)

    #test results
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [0]:
rnn(x_train, y_train, x_test, y_test)


Train on 543915 samples, validate on 135979 samples
Epoch 1/10
 30600/543915 [>.............................] - ETA: 38:40 - loss: 0.5691 - acc: 0.7144

KeyboardInterrupt: ignored